In [1]:
# Training: 2020-02-15 to 2020-05-11
# Test: 2020-05-12 to 2020-06-10
# With SMA, the training window is a rolling window

# Countries to work with:
# - Canada
# - Japan
# - Italy
# - UK
# - Germany 
# - France

In [2]:
%python
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
display(spark.sql("SELECT * FROM g6_confirmed_cases"))

Date,canada_cases,japan_cases,italy_cases,uk_cases,germany_cases,france_cases
2020-02-15T00:00:00.000+0000,0,14,0,0,0,1
2020-02-16T00:00:00.000+0000,0,16,0,0,0,0
2020-02-17T00:00:00.000+0000,1,7,0,0,0,0
2020-02-18T00:00:00.000+0000,0,8,0,0,0,0
2020-02-19T00:00:00.000+0000,0,10,0,0,0,0
2020-02-20T00:00:00.000+0000,0,10,0,0,0,0
2020-02-21T00:00:00.000+0000,1,11,17,0,0,0
2020-02-22T00:00:00.000+0000,0,17,42,0,0,0
2020-02-23T00:00:00.000+0000,0,25,93,0,0,0
2020-02-24T00:00:00.000+0000,1,12,74,4,0,0


In [4]:
%python

# This can be used for the g6_confirmed_cases dataset
def sma(Days, Lag, Country, Prediction_Start):
  Country_Query = spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY Date ASC) AS Row, Date, " + Country + ", lag(MA10," + str(Lag) + ") over(ORDER BY Date) AS Prediction FROM (SELECT " + Country + ", Date, avg(" + Country + ") over(ORDER BY Date ASC ROWS " + str(Days-1) + " PRECEDING) AS MA10 FROM g6_confirmed_cases)")
  Country_Query_Prediction = Country_Query.select('Row', 'Date', Country, 'Prediction').where(Country_Query['Row'] > Prediction_Start)
  #display(Country_Query_Prediction)
  return Country_Query_Prediction

# This can be used for the g6_confirmed_cases dataset for plotting
def sma_plot(Days, Lag, Country, Prediction_Start):
  Country_Query = spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY Date ASC) AS Row, Date, " + Country + ", lag(MA10," + str(Lag) + ") over(ORDER BY Date) AS Prediction FROM (SELECT " + Country + ", Date, avg(" + Country + ") over(ORDER BY Date ASC ROWS " + str(Days-1) + " PRECEDING) AS MA10 FROM g6_confirmed_cases)")
  Country_Query_Prediction = Country_Query.select('Row', 'Date', Country, 'Prediction').where(Country_Query['Row'] > Prediction_Start)
  display(Country_Query_Prediction)
  return Country_Query_Prediction

In [5]:
canada_1_1 = sma_plot(1, 1, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1173.0
89,2020-05-13T00:00:00.000+0000,1149,1155.0
90,2020-05-14T00:00:00.000+0000,1213,1149.0
91,2020-05-15T00:00:00.000+0000,1178,1213.0
92,2020-05-16T00:00:00.000+0000,1247,1178.0
93,2020-05-17T00:00:00.000+0000,1126,1247.0
94,2020-05-18T00:00:00.000+0000,1079,1126.0
95,2020-05-19T00:00:00.000+0000,1082,1079.0
96,2020-05-20T00:00:00.000+0000,1082,1082.0
97,2020-05-21T00:00:00.000+0000,1167,1082.0


In [6]:
canada_7_1 = sma_plot(7, 1, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1329.5714285714287
89,2020-05-13T00:00:00.000+0000,1149,1314.857142857143
90,2020-05-14T00:00:00.000+0000,1213,1267.7142857142858
91,2020-05-15T00:00:00.000+0000,1178,1225.7142857142858
92,2020-05-16T00:00:00.000+0000,1247,1183.5714285714287
93,2020-05-17T00:00:00.000+0000,1126,1184.0
94,2020-05-18T00:00:00.000+0000,1079,1177.2857142857142
95,2020-05-19T00:00:00.000+0000,1082,1163.857142857143
96,2020-05-20T00:00:00.000+0000,1082,1153.4285714285713
97,2020-05-21T00:00:00.000+0000,1167,1143.857142857143


In [7]:
canada_30_1 = sma_plot(30, 1, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1552.7666666666667
89,2020-05-13T00:00:00.000+0000,1149,1558.5
90,2020-05-14T00:00:00.000+0000,1213,1550.7666666666667
91,2020-05-15T00:00:00.000+0000,1178,1546.0333333333333
92,2020-05-16T00:00:00.000+0000,1247,1546.1666666666667
93,2020-05-17T00:00:00.000+0000,1126,1520.2333333333333
94,2020-05-18T00:00:00.000+0000,1079,1490.9333333333334
95,2020-05-19T00:00:00.000+0000,1082,1475.5
96,2020-05-20T00:00:00.000+0000,1082,1469.0
97,2020-05-21T00:00:00.000+0000,1167,1437.5666666666666


In [8]:
canada_1_7 = sma(1, 7, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1258.0
89,2020-05-13T00:00:00.000+0000,1149,1479.0
90,2020-05-14T00:00:00.000+0000,1213,1507.0
91,2020-05-15T00:00:00.000+0000,1178,1473.0
92,2020-05-16T00:00:00.000+0000,1247,1244.0
93,2020-05-17T00:00:00.000+0000,1126,1173.0
94,2020-05-18T00:00:00.000+0000,1079,1173.0
95,2020-05-19T00:00:00.000+0000,1082,1155.0
96,2020-05-20T00:00:00.000+0000,1082,1149.0
97,2020-05-21T00:00:00.000+0000,1167,1213.0


In [9]:
canada_7_7 = sma_plot(7, 7, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1644.4285714285713
89,2020-05-13T00:00:00.000+0000,1149,1610.7142857142858
90,2020-05-14T00:00:00.000+0000,1213,1598.5714285714287
91,2020-05-15T00:00:00.000+0000,1178,1539.7142857142858
92,2020-05-16T00:00:00.000+0000,1247,1491.2857142857142
93,2020-05-17T00:00:00.000+0000,1126,1369.5714285714287
94,2020-05-18T00:00:00.000+0000,1079,1329.5714285714287
95,2020-05-19T00:00:00.000+0000,1082,1314.857142857143
96,2020-05-20T00:00:00.000+0000,1082,1267.7142857142858
97,2020-05-21T00:00:00.000+0000,1167,1225.7142857142858


In [10]:
canada_30_7 = sma_plot(30, 7, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1536.4666666666667
89,2020-05-13T00:00:00.000+0000,1149,1558.8666666666666
90,2020-05-14T00:00:00.000+0000,1213,1565.4666666666667
91,2020-05-15T00:00:00.000+0000,1178,1572.2666666666667
92,2020-05-16T00:00:00.000+0000,1247,1563.3
93,2020-05-17T00:00:00.000+0000,1126,1555.5666666666666
94,2020-05-18T00:00:00.000+0000,1079,1552.7666666666667
95,2020-05-19T00:00:00.000+0000,1082,1558.5
96,2020-05-20T00:00:00.000+0000,1082,1550.7666666666667
97,2020-05-21T00:00:00.000+0000,1167,1546.0333333333333


In [11]:
canada_1_30 = sma_plot(1, 30, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,983.0
89,2020-05-13T00:00:00.000+0000,1149,1381.0
90,2020-05-14T00:00:00.000+0000,1213,1355.0
91,2020-05-15T00:00:00.000+0000,1178,1174.0
92,2020-05-16T00:00:00.000+0000,1247,2025.0
93,2020-05-17T00:00:00.000+0000,1126,2005.0
94,2020-05-18T00:00:00.000+0000,1079,1542.0
95,2020-05-19T00:00:00.000+0000,1082,1277.0
96,2020-05-20T00:00:00.000+0000,1082,2025.0
97,2020-05-21T00:00:00.000+0000,1167,1744.0


In [12]:
canada_7_30 = sma(7, 30, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,1220.4285714285713
89,2020-05-13T00:00:00.000+0000,1149,1302.4285714285713
90,2020-05-14T00:00:00.000+0000,1213,1309.0
91,2020-05-15T00:00:00.000+0000,1178,1295.4285714285713
92,2020-05-16T00:00:00.000+0000,1247,1368.5714285714287
93,2020-05-17T00:00:00.000+0000,1126,1454.2857142857142
94,2020-05-18T00:00:00.000+0000,1079,1495.0
95,2020-05-19T00:00:00.000+0000,1082,1537.0
96,2020-05-20T00:00:00.000+0000,1082,1629.0
97,2020-05-21T00:00:00.000+0000,1167,1684.5714285714287


In [13]:
canada_30_30 = sma_plot(30, 30, "canada_cases", 87)

Row,Date,canada_cases,Prediction
88,2020-05-12T00:00:00.000+0000,1155,778.0666666666667
89,2020-05-13T00:00:00.000+0000,1149,823.9333333333333
90,2020-05-14T00:00:00.000+0000,1213,867.3
91,2020-05-15T00:00:00.000+0000,1178,901.0
92,2020-05-16T00:00:00.000+0000,1247,966.6
93,2020-05-17T00:00:00.000+0000,1126,1027.5
94,2020-05-18T00:00:00.000+0000,1079,1074.1333333333334
95,2020-05-19T00:00:00.000+0000,1082,1111.9666666666667
96,2020-05-20T00:00:00.000+0000,1082,1168.3333333333333
97,2020-05-21T00:00:00.000+0000,1167,1220.1666666666667


In [14]:
%python

import pyspark.sql.functions as func

def SMAPE(Country_Query_Prediction, Country):
  SMAPE = Country_Query_Prediction.withColumn("SMAPE_num", abs(col("Prediction")-col(Country)))
  SMAPE = SMAPE.withColumn("SMAPE_den", (abs(col("Prediction"))+abs(col(Country)))/2)
  SMAPE = SMAPE.withColumn("SMAPE_n", col("SMAPE_num")/col("SMAPE_den"))
  SMAPE_value = SMAPE.groupby().sum("SMAPE_n").collect()[0][0]/SMAPE.count()*100
  return SMAPE_value

def SMAPE_table(Country):
  # Different parameters for moving average
  result_1_1 = sma(1, 1, Country, 87)
  result_7_1 = sma(7, 1, Country, 87)
  result_14_1 = sma(14, 1, Country, 87)
  result_30_1 = sma(30, 1, Country, 87)
  result_1_7 = sma(1, 7, Country, 87)
  result_7_7 = sma(7, 7, Country, 87)
  result_14_7 = sma(14, 7, Country, 87)
  result_30_7 = sma(30, 7, Country, 87)
  result_1_14 = sma(1, 14, Country, 87)
  result_7_14 = sma(7, 14, Country, 87)
  result_14_14 = sma(14, 14, Country, 87)
  result_30_14 = sma(30, 14, Country, 87)
  result_1_30 = sma(1, 30, Country, 87)
  result_7_30 = sma(7, 30, Country, 87)
  result_14_30 = sma(14, 30, Country, 87)
  result_30_30 = sma(30, 30, Country, 87)

  # Creating a dataframe to make a table to summarize the results
  Results = Row("Model", "1 Day", "7 Day", "14 Day", "30 Day")
  results1 = Results("1 Day Prediction", SMAPE(result_1_1, Country), SMAPE(result_7_1, Country), SMAPE(result_14_1, Country), SMAPE(result_30_1, Country))
  results2 = Results("7 Day Prediction", SMAPE(result_1_7, Country), SMAPE(result_7_7, Country), SMAPE(result_14_7, Country), SMAPE(result_30_7, Country))
  results3 = Results("14 Day Prediction", SMAPE(result_1_14, Country), SMAPE(result_7_14, Country), SMAPE(result_14_14, Country), SMAPE(result_30_14, Country))
  results4 = Results("30 Day Prediction", SMAPE(result_1_30, Country), SMAPE(result_7_30, Country), SMAPE(result_14_30, Country), SMAPE(result_30_30, Country))
  df = spark.createDataFrame([results1, results2, results3, results4])
  df = df.withColumn(
    "1 Day", func.round(df["1 Day"], 2)).withColumn(
    "7 Day", func.round(df["7 Day"],2)).withColumn(
    "14 Day", func.round(df["14 Day"],2)).withColumn(
    "30 Day", func.round(df["30 Day"],2))
  print(Country)
  df.show()

In [15]:
#RMSE Functions

def RMSE(Country_Query_Prediction, Country):
  RMSE = Country_Query_Prediction.withColumn("RMSE_num", (col("Prediction")-col(Country))**2)
  RMSE_value = ((RMSE.groupby().sum("RMSE_num").collect()[0][0]/RMSE.count())**0.5)
  return RMSE_value

def RMSE_table(Country):
  # Different parameters for moving average
  result_1_1 = sma(1, 1, Country, 87)
  result_7_1 = sma(7, 1, Country, 87)
  result_14_1 = sma(14, 1, Country, 87)
  result_30_1 = sma(30, 1, Country, 87)
  result_1_7 = sma(1, 7, Country, 87)
  result_7_7 = sma(7, 7, Country, 87)
  result_14_7 = sma(14, 7, Country, 87)
  result_30_7 = sma(30, 7, Country, 87)
  result_1_14 = sma(1, 14, Country, 87)
  result_7_14 = sma(7, 14, Country, 87)
  result_14_14 = sma(14, 14, Country, 87)
  result_30_14 = sma(30, 14, Country, 87)
  result_1_30 = sma(1, 30, Country, 87)
  result_7_30 = sma(7, 30, Country, 87)
  result_14_30 = sma(14, 30, Country, 87)
  result_30_30 = sma(30, 30, Country, 87)

  # Creating a dataframe to make a table to summarize the results
  Results = Row("Model", "1 Day", "7 Day", "14 Day", "30 Day")
  results1 = Results("1 Day Prediction", RMSE(result_1_1, Country), RMSE(result_7_1, Country), RMSE(result_14_1, Country), RMSE(result_30_1, Country))
  results2 = Results("7 Day Prediction", RMSE(result_1_7, Country), RMSE(result_7_7, Country), RMSE(result_14_7, Country), RMSE(result_30_7, Country))
  results3 = Results("14 Day Prediction", RMSE(result_1_14, Country), RMSE(result_7_14, Country), RMSE(result_14_14, Country), RMSE(result_30_14, Country))
  results4 = Results("30 Day Prediction", RMSE(result_1_30, Country), RMSE(result_7_30, Country), RMSE(result_14_30, Country), RMSE(result_30_30, Country))
  df = spark.createDataFrame([results1, results2, results3, results4])
  df = df.withColumn(
    "1 Day", func.round(df["1 Day"], 2)).withColumn(
    "7 Day", func.round(df["7 Day"],2)).withColumn(
    "14 Day", func.round(df["14 Day"],2)).withColumn(
    "30 Day", func.round(df["30 Day"],2))
  print(Country)
  df.show()


In [16]:
RMSE_table("canada_cases")
RMSE_table("japan_cases")
RMSE_table("italy_cases")
RMSE_table("uk_cases")
RMSE_table("germany_cases")
RMSE_table("france_cases")

canada_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|30 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 91.77|127.36|221.28|390.38|
 7 Day Prediction|206.93|298.11|369.06|505.48|
14 Day Prediction|418.52|452.64|516.97|597.21|
30 Day Prediction|711.18|708.95|705.56|618.11|
+-----------------+------+------+------+------+

japan_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|30 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 30.15| 24.99| 43.84| 136.2|
 7 Day Prediction| 43.98| 58.84|107.35|201.29|
14 Day Prediction|108.06|157.93|196.61|266.03|
30 Day Prediction|356.62|351.06|353.48|304.17|
+-----------------+------+------+------+------+

italy_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 30 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| 215.19| 176.37| 282.87| 777.79|
 7 Day Prediction| 274.82| 366.27| 574.95|1189.72|
14 Day Prediction| 608.94| 789.55|1071.56|1740.51|
30 Day Prediction|1860.91|2131.12|2432.37|3121.48|
+-----------------+-------+-------+-------+-------+

uk_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 30 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction| 856.31| 810.18|1032.66|1563.63|
 7 Day Prediction| 1205.5|1259.42|1493.51|1985.35|
14 Day Prediction| 1805.5|1787.24|2007.09|2343.23|
30 Day Prediction|2714.48|2723.66| 2704.3|2459.85|
+-----------------+-------+-------+-------+-------+

germany_cases
+-----------------+------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 30 Day|
+-----------------+------+-------+-------+-------+
 1 Day Prediction|325.27| 248.43| 265.58| 523.03|
 7 Day Prediction|318.17| 287.22| 382.75| 830.12|
14 Day Prediction|444.14| 494.69| 679.86| 1388.7|
30 Day Prediction|1350.5|1648.81|2146.56|2883.48|
+-----------------+------+-------+-------+-------+

france_cases
+-----------------+-------+-------+-------+-------+
 Model| 1 Day| 7 Day| 14 Day| 30 Day|
+-----------------+-------+-------+-------+-------+
 1 Day Prediction|1132.97| 986.4|1013.27|1157.26|
 7 Day Prediction|1327.69|1096.76|1132.27|1433.61|
14 Day Prediction|1273.17|1240.34|1348.45| 1823.6|
30 Day Prediction| 2327.8|2307.44|2476.79|2611.99|
+-----------------+-------+-------+-------+-------+

In [17]:
SMAPE_table("canada_cases")
SMAPE_table("japan_cases")
SMAPE_table("italy_cases")
SMAPE_table("UK_cases")
SMAPE_table("germany_cases")
SMAPE_table("france_cases")

canada_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|30 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction| 8.5|13.14| 21.93| 37.04|
 7 Day Prediction|19.41|28.02| 34.78| 45.34|
14 Day Prediction|35.87|41.01| 46.09| 50.15|
30 Day Prediction|53.89|53.94| 51.86| 46.7|
+-----------------+-----+-----+------+------+

japan_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|30 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 56.01| 38.87| 50.33| 94.05|
 7 Day Prediction| 57.29| 55.48| 72.02|120.67|
14 Day Prediction| 76.08| 84.73|104.87|142.54|
30 Day Prediction|137.51|151.49|156.07|149.91|
+-----------------+------+------+------+------+

italy_cases
+-----------------+------+-----+------+------+
 Model| 1 Day|7 Day|14 Day|30 Day|
+-----------------+------+-----+------+------+
 1 Day Prediction| 30.68|27.01| 41.5| 82.59|
 7 Day Prediction| 36.65|50.73| 68.09|104.73|
14 Day Prediction| 66.29|81.78| 97.16|124.93|
30 Day Prediction|124.25|133.6|140.67|151.33|
+-----------------+------+-----+------+------+

UK_cases
+-----------------+-----+-----+------+------+
 Model|1 Day|7 Day|14 Day|30 Day|
+-----------------+-----+-----+------+------+
 1 Day Prediction|27.07|25.26| 33.67| 52.76|
 7 Day Prediction|37.13|40.36| 48.23| 63.41|
14 Day Prediction|54.34|55.34| 62.84| 70.62|
30 Day Prediction|74.75|76.83| 76.02| 67.53|
+-----------------+-----+-----+------+------+

germany_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|30 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 49.94| 43.28| 46.04| 68.46|
 7 Day Prediction| 50.03| 47.54| 58.15| 88.8|
14 Day Prediction| 60.56| 68.03| 79.12|112.79|
30 Day Prediction|105.78|119.59|132.04|150.54|
+-----------------+------+------+------+------+

france_cases
+-----------------+------+------+------+------+
 Model| 1 Day| 7 Day|14 Day|30 Day|
+-----------------+------+------+------+------+
 1 Day Prediction| 87.36| 82.75| 84.4|105.86|
 7 Day Prediction| 87.05| 89.51| 96.83|114.65|
14 Day Prediction| 95.75|100.35|106.43|124.52|
30 Day Prediction|129.54|132.61|136.09|140.02|
+-----------------+------+------+------+------+

In [18]:
%python
#Test Queries for Canada Dataset
#display(spark.sql("SELECT canada_cases, Date, avg(canada_cases) over(ORDER BY Date ASC ROWS " + str(3-1) + " PRECEDING) AS MA10 FROM g6_confirmed_cases"))
#display(spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY Date ASC) AS Row, Date, canada_cases, lag(MA10,1) over(ORDER BY Date) AS Prediction FROM (SELECT canada_cases, Date, avg(canada_cases) over(ORDER BY Date ASC ROWS 2 PRECEDING) AS MA10 FROM g6_confirmed_cases)"))

# This can be used for the covid_cases dataset
# def sma_cc(Days, Country, Prediction_Start):
#   Country_Query = spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY Date ASC) AS Row, Country_Region, Date, TargetValue, lag(MA10,1) over(ORDER BY Date) AS Prediction FROM (SELECT Country_Region, Date, TargetValue, avg(TargetValue) over(ORDER BY Date ASC ROWS " + str(Days-1) + " PRECEDING) AS MA10 FROM covid_cases WHERE " + Country + " and Target = 'ConfirmedCases')")
#   Country_Query_Prediction = Country_Query.select('Row', 'Country_Region', 'Date', 'TargetValue', 'Prediction').where(Country_Query['Row'] > Prediction_Start)
#   display(Country_Query_Prediction)
#   return Country_Query_Prediction

# def SMAPE_cc(Country_Query_Prediction):
#   SMAPE = Country_Query_Prediction.withColumn("SMAPE_num", abs(col("Prediction")-col("TargetValue")))
#   SMAPE = SMAPE.withColumn("SMAPE_den", (abs(col("Prediction"))+abs(col("TargetValue")))/2)
#   SMAPE = SMAPE.withColumn("SMAPE_n", col("SMAPE_num")/col("SMAPE_den"))
#   overCategory = Window.partitionBy("Country_Region")
#   SMAPE = SMAPE.withColumn(
#     "SMAPE_n", (sum(col("SMAPE_num")/col("SMAPE_den")).over(overCategory))).withColumn(
#     "SMAPE_count", (count(col("SMAPE_num")).over(overCategory))).withColumn(
#     "SMAPE", col("SMAPE_n") / col("SMAPE_count")).select(
#     "Country_Region", "TargetValue", "Prediction", "SMAPE_num", "SMAPE_den", "SMAPE_n", "SMAPE_count", "SMAPE")
#   SMAPE_value = SMAPE.select("SMAPE").distinct().collect()[0]
#   #SMAPE.show()
#   return SMAPE_value

# Using the covid_cases dataset
# Canada_Prediction_1 = sma(1, "Country_Region = 'Canada' AND Province_State = 'Ontario'", 71)
# Canada_Prediction_7 = sma(7, "Country_Region = 'Canada' AND Province_State = 'Ontario'", 71)
# Canada_Prediction_30 = sma(30, "Country_Region = 'Canada' AND Province_State = 'Ontario'", 71)